# Catastrophe Modeling Example

This example demonstrates catastrophe modeling using the Proteus Actuarial Library (PAL).

The example shows how to:
- Load catastrophe year loss table (YLT) data
- Upsample the data to match simulation requirements
- Apply inflation modeling
- Calculate recoveries with limits
- Visualize the results with CDF plots

## Setup and Configuration

In [12]:
import os

# Enable plotting in Jupyter notebooks
# The development container sets PAL_SUPPRESS_PLOTS=true by default to prevent
# plots from appearing during automated testing and CI/CD runs. However, we want
# to see plots when running notebooks interactively, so we override it here.
os.environ["PAL_SUPPRESS_PLOTS"] = "false"

In [13]:
import math

import pandas as pd  # type: ignore
from pal import config, distributions
from pal._maths import xp as np
from pal.variables import FreqSevSims, ProteusVariable, StochasticScalar

In [14]:
# Configure simulation parameters
n_sims = 100000
config.n_sims = n_sims

# Define lines of business
lobs = ["Motor", "Property", "Liability", "Marine", "Aviation"]
print(f"Lines of business: {lobs}")
print(f"Number of simulations: {n_sims:,}")

Lines of business: ['Motor', 'Property', 'Liability', 'Marine', 'Aviation']
Number of simulations: 100,000


## Load and Process Catastrophe Data

In [15]:
# Load the catastrophe year loss table (YLT)
df = pd.read_csv("../data/catastrophes/cat_ylt.csv", index_col=0)  # type: ignore[misc]
print(f"Loaded catastrophe data with shape: {df.shape}")
print("\nFirst few rows:")
df.head()

Loaded catastrophe data with shape: (20049, 6)

First few rows:


,sim,Motor,Property,Liability,Marine,Aviation
id,,,,,,
10000000,1,0.0,8.198728e+07,0.0,7.038996e+06,1.553890e+06
10000001,1,0.0,2.139836e+07,0.0,1.837151e+06,4.055592e+05
10000002,1,0.0,9.299629e+07,0.0,7.984172e+06,1.762542e+06
10000003,1,0.0,1.031189e+07,0.0,8.853243e+05,1.954393e+05
10000004,1,0.0,5.328804e+06,0.0,4.575030e+05,1.009958e+05


In [ ]:
# Upsample the catastrophe YLTs to match our simulation requirements
ylt_sims = 10000
up_sample_factor = math.ceil(n_sims / ylt_sims)

print(f"Original YLT simulations: {ylt_sims:,}")
print(f"Target simulations: {n_sims:,}")
print(f"Upsampling factor: {up_sample_factor}")

# Create upsampled simulation indices and values
# Convert pandas values to numpy array with explicit dtype for better type checking
sim_index = np.array(
    df["sim"].values,  # type: ignore[reportUnknownMemberType]
    dtype=int,
).repeat(up_sample_factor)
print(f"\nUpsampled simulation index length: {len(sim_index):,}")

Original YLT simulations: 10,000
Target simulations: 100,000
Upsampling factor: 10

Upsampled simulation index length: 200,490


## Create Catastrophe Loss Variables

In [ ]:
# Create ProteusVariable containing FreqSevSims for each line of business
cat_losses = ProteusVariable(
    "lob",
    {
        lob: FreqSevSims(
            sim_index,
            df[lob].values.repeat(up_sample_factor),  # type: ignore[reportUnknownMemberType]
            n_sims=config.n_sims,
        )
        for lob in lobs
    },
)

print("Created catastrophe loss variables for all lines of business")
print(f"Variable dimension: {cat_losses.dim_name}")
print(f"Lines of business: {list(cat_losses.values.keys())}")

Created catastrophe loss variables for all lines of business
Variable dimension: lob
Lines of business: ['Motor', 'Property', 'Liability', 'Marine', 'Aviation']


## Apply Inflation Modeling

In [18]:
# Create inflation rate variable (Normal distribution with 5% mean, 2% std dev)
inflation_rate = distributions.Normal(0.05, 0.02).generate()

print("Inflation rate statistics:")
print(f"  Mean: {np.mean(inflation_rate):.3f}")
print(f"  Std Dev: {np.std(inflation_rate):.3f}")
print(f"  Min: {np.min(inflation_rate):.3f}")
print(f"  Max: {np.max(inflation_rate):.3f}")

Inflation rate statistics:
  Mean: 0.050
  Std Dev: 0.020
  Min: -0.052
  Max: 0.145


In [19]:
# Apply inflation to catastrophe losses
scaled_cat_losses_by_lob = cat_losses * (1 + inflation_rate)

print(
    "Applied inflation scaling to catastrophe losses.\n"
    "Scaled losses by line of business: "
    f"{list(scaled_cat_losses_by_lob.values.keys())}"
)

Applied inflation scaling to catastrophe losses.
Scaled losses by line of business: ['Motor', 'Property', 'Liability', 'Marine', 'Aviation']


In [20]:
# Sum across all lines of business to get total catastrophe losses
# Use ProteusVariable.sum() method to properly aggregate FreqSevSims
scaled_cat_losses: FreqSevSims = scaled_cat_losses_by_lob.sum()

# Aggregate individual events to get total loss per simulation
aggregate_losses: StochasticScalar = scaled_cat_losses.aggregate()

print("Total scaled catastrophe losses statistics:")
print(f"  Mean: {np.mean(aggregate_losses):,.0f}")
print(f"  Std Dev: {np.std(aggregate_losses):,.0f}")
print(f"  Min: {np.min(aggregate_losses):,.0f}")
print(f"  Max: {np.max(aggregate_losses):,.0f}")

Total scaled catastrophe losses statistics:
  Mean: 78,572,231
  Std Dev: 421,424,666
  Min: 0
  Max: 16,477,085,395


## Calculate Recoveries

In [21]:
# Calculate recoveries with limits
# Recoveries = min(max(losses - 10M, 0), 10M)
# This represents a reinsurance layer: 10M xs 10M (10M excess of 10M)
recoveries_fs: FreqSevSims = np.minimum(
    np.maximum(scaled_cat_losses - 10000000, 0), 10000000
)

# Aggregate recoveries to get total recovery per simulation
recoveries: StochasticScalar = recoveries_fs.aggregate()

print("Recoveries statistics:")
print(f"  Mean: {np.mean(recoveries):,.0f}")
print(f"  Std Dev: {np.std(recoveries):,.0f}")
print(f"  Min: {np.min(recoveries):,.0f}")
print(f"  Max: {np.max(recoveries):,.0f}")

# Split long line for readability
non_zero = (recoveries.values > 0).sum()
total = len(recoveries.values)
print(f"  Non-zero recoveries: {non_zero:,} out of {total:,}")

Recoveries statistics:
  Mean: 11,299,496
  Std Dev: 49,136,608
  Min: 0
  Max: 700,000,000
  Non-zero recoveries: 7,010 out of 100,000


## Visualize Results

In [22]:
# Show cumulative distribution function (CDF) of recoveries
recoveries.show_cdf()